In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('predykcja').getOrCreate()
data = spark.read.csv(r"C:\Users\Asus\Downloads\train_student.csv", inferSchema = True, header=True)

In [7]:
data.printSchema

<bound method DataFrame.printSchema of DataFrame[BirthYear: int, Sex: int, LicenseYear: double, CarBrand: int, CarType: int, CarYear: int, CarEngine: int, EngineCap: int, CarValue: int, AssistanceYears: double, Accidents: int, NextAccident: int]>

In [3]:
from pyspark.sql.functions import col, mean

In [19]:
data.describe

<bound method DataFrame.describe of DataFrame[BirthYear: int, Sex: int, LicenseYear: double, CarBrand: int, CarType: int, CarYear: int, CarEngine: int, EngineCap: int, CarValue: int, AssistanceYears: double, Accidents: int, NextAccident: int, DriverAge: int, CarAge: int]>

In [6]:
data = data.withColumn('EngineCap', data['EngineCap'].astype('int'))

In [8]:
columns = data.columns

In [9]:
columns

['BirthYear',
 'Sex',
 'LicenseYear',
 'CarBrand',
 'CarType',
 'CarYear',
 'CarEngine',
 'EngineCap',
 'CarValue',
 'AssistanceYears',
 'Accidents',
 'NextAccident']

In [10]:
current = 2024
data = data.withColumn('DriverAge', current - data['BirthYear'])
data = data.withColumn("CarAge", current - data['CarYear'])

In [11]:
from pyspark.ml.feature import VectorAssembler, Imputer
for column in columns:
    imputer = Imputer(inputCols=[column], outputCols=[column])
    model = imputer.fit(data)
    data = model.transform(data)

In [12]:
pandas_df = data.select(columns).toPandas()

In [30]:
columns

['BirthYear',
 'Sex',
 'LicenseYear',
 'CarBrand',
 'CarType',
 'CarYear',
 'CarEngine',
 'EngineCap',
 'CarValue',
 'AssistanceYears',
 'Accidents',
 'NextAccident',
 'DriverAge',
 'CarAge']

In [13]:
import matplotlib.pyplot as plt



In [32]:
!pip install seaborn

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/294.9 kB 330.3 kB/s eta 0:00:01
   ---- ---------------------------------- 30.7/294.9 kB 330.3 kB/s eta 0:00:01
   --------- ----------------------------- 71.7/294.9 kB 491.5 kB/s eta 0:00:01
   ------------------------------ --------- 225.3/294.9 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 1.1 MB/s eta 0:00:00


In [36]:
columns

['BirthYear',
 'Sex',
 'LicenseYear',
 'CarBrand',
 'CarType',
 'CarYear',
 'CarEngine',
 'EngineCap',
 'CarValue',
 'AssistanceYears',
 'Accidents',
 'NextAccident',
 'DriverAge',
 'CarAge']

In [15]:
correlation_matrix = data.corr('CarAge', 'NextAccident')

print(correlation_matrix)


0.08079373302229692


In [41]:
import statsmodels.api as sm

In [78]:
X = pandas_df[[
'BirthYear',
 'Sex',
 'LicenseYear',
 'CarBrand',
 'CarType',
 'CarYear',
 'CarEngine',
 'EngineCap',
 'CarValue',
 'AssistanceYears',
 'Accidents',
 'DriverAge',
 'CarAge'
]]

In [79]:
Y = pandas_df['NextAccident']

In [80]:
X = sm.add_constant(X)
logit_model = sm.Logit(Y, X)

# Dopasowanie modelu do danych
logit_result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.441637
         Iterations 13


In [81]:
logit_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           NextAccident   No. Observations:                80000
Model:                          Logit   Df Residuals:                    79986
Method:                           MLE   Df Model:                           13
Date:                Tue, 02 Apr 2024   Pseudo R-squ.:                 0.01817
Time:                        17:36:44   Log-Likelihood:                -35331.
converged:                       True   LL-Null:                       -35985.
Covariance Type:            nonrobust   LLR p-value:                1.261e-271
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            2.619e-05   4.11e+05   6.38e-11      1.000   -8.05e+05    8.05e+05
BirthYear           0.0005      0.087      0.006      0.995      -0.171       0.172
Sex                 0.0030      0.020      0.150      0.880      -0.036       0.042
LicenseYear        -0.0044      0.002     -2.044      0.041      -0.009      -0.000
CarBrand           -0.0001      0.004     -0.030      0.976      -0.008       0.007
CarType            -0.0059      0.010     -0.567      0.571      -0.026       0.014
CarYear            -0.0010    202.916  -4.91e-06      1.000    -397.709     397.707
CarEngine           0.0003   4.11e-05      7.848      0.000       0.000       0.000
EngineCap           0.0008      0.000      3.177      0.001       0.000       0.001
CarValue         -2.46e-05      1e-06    -24.537      0.000   -2.66e-05   -2.26e-05
AssistanceYears  5.247e-05      0.003      0.020      0.984      -0.005       0.005
Accidents           0.0071      0.009      0.755      0.450      -0.011       0.025
DriverAge          -0.0010      0.087     -0.012      0.991      -0.172       0.170
CarAge             -0.0006    202.916  -2.89e-06      1.000    -397.709     397.708
===================================================================================
"""